<a href="https://colab.research.google.com/github/PratikStar/google-colab/blob/main/2_2_Timbre_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


### Install tensorflow v2.4.0

In [2]:
!pip uninstall --yes tensorflow
!pip install tensorflow==2.4.0

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 394.7 MB 15 kB/s 
     |████████████████████████████████| 2.9 MB 74.1 MB/s 
     |████████████████████████████████| 462 kB 67.1 MB/s 
     |████████████████████████████████| 3.8 MB 24.2 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
    Uninstalling grpcio-1.40.0:
      Successfully uninstalled grpcio-1.40.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.

## Autoencoder Code

In [3]:
"#@title"
import os
import pickle
import csv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Flatten, Dense, Reshape, Conv2DTranspose, Activation, Lambda, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import numpy as np
import tensorflow as tf

print(tf.__version__)
tf.compat.v1.disable_eager_execution()


class VAE:
    """
    VAE represents a Deep Convolutional variational autoencoder architecture
    with mirrored encoder and decoder components.
    """

    def __init__(self,
                 timbre_input_shape,
                 music_input_shape,
                 conv_filters,
                 conv_kernels,
                 conv_strides,
                 music_latent_space_dim,
                 timbre_latent_space_dim):
        self.timbre_input_shape = timbre_input_shape
        self.music_input_shape = music_input_shape

        self.conv_filters = conv_filters
        self.conv_kernels = conv_kernels 
        self.conv_strides = conv_strides 
        self.music_latent_space_dim = music_latent_space_dim 
        self.timbre_latent_space_dim = timbre_latent_space_dim 
        self.reconstruction_loss_weight = 1000000

        self.encoder = None
        self.decoder = None
        self.model = None

        self._num_conv_layers = len(conv_filters)
        self._shape_before_bottleneck = None
        self._model_input = None

        self._build()

    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        self.model.summary()

    def compile(self, learning_rate=0.0001):
        optimizer = Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss=self._calculate_combined_loss,
                           metrics=[self._calculate_reconstruction_loss,
                                    self._calculate_kl_loss])

    def train(self, generator, dataset_length, batch_size, num_epochs, callbacks):
        if dataset_length % batch_size != 0:
            raise Exception("dataset_length % batch_size = " + str(dataset_length % batch_size) + " != 0. Exiting!!")
        return self.model.fit(x=generator,
                       steps_per_epoch= dataset_length/batch_size,
                       epochs=num_epochs,
                       callbacks=callbacks
              )

    def save(self, save_folder="."):
        self._create_folder_if_it_doesnt_exist(save_folder)
        self.save_parameters(save_folder)
        self._save_weights(save_folder)

    def reconstruct(self, images):
        latent_representations = self.encoder.predict(images)
        reconstructed_images = self.decoder.predict(latent_representations)
        return reconstructed_images, latent_representations

    def save_parameters(self, save_folder):
        self._create_folder_if_it_doesnt_exist(save_folder)
        parameters = [
            self.timbre_input_shape,
            self.music_input_shape,
            self.conv_filters,
            self.conv_kernels,
            self.conv_strides,
            self.music_latent_space_dim,
            self.timbre_latent_space_dim
        ]
        save_path = os.path.join(save_folder, "parameters.pkl")
        with open(save_path, "wb") as f:
            print(parameters)
            pickle.dump(parameters, f)

    def _save_weights(self, save_folder):
        save_path = os.path.join(save_folder, "weights.h5")
        self.model.save_weights(save_path)

    @classmethod
    def load(cls, save_folder=".", weights_file_name="weights.h5"):
        parameters_path = os.path.join(save_folder, "parameters.pkl")
        weights_path = os.path.join(save_folder, weights_file_name)
        if not os.path.exists(parameters_path) or not os.path.exists(weights_path):
            return None
        with open(parameters_path, "rb") as f:
            parameters = pickle.load(f)
        autoencoder = VAE(*parameters)
        autoencoder.load_weights(weights_path)
        return autoencoder

    def load_weights(self, weights_path):
        self.model.load_weights(weights_path)

    def _calculate_combined_loss(self, y_target, y_predicted):
        reconstruction_loss = self._calculate_reconstruction_loss(y_target, y_predicted)
        kl_loss = self._calculate_kl_loss(y_target, y_predicted)
        combined_loss = self.reconstruction_loss_weight * reconstruction_loss\
                                                         + kl_loss
        return combined_loss

    def _calculate_reconstruction_loss(self, y_target, y_predicted):
        error = y_target - y_predicted
        reconstruction_loss = K.mean(K.square(error), axis=[1, 2, 3])
        return reconstruction_loss

    def _calculate_kl_loss(self, y_target, y_predicted):
        kl_loss = -0.5 * K.sum(1 + self.log_variance - K.square(self.mu) -
                               K.exp(self.log_variance), axis=1)
        return kl_loss

    def _create_folder_if_it_doesnt_exist(self, folder):
        if not os.path.exists(folder):
            os.makedirs(folder)

    def _build(self):
        self._build_encoder()
        self._build_decoder()
        self._build_autoencoder()

    def _build_autoencoder(self):
        model_input = self._model_input
        model_output = self.decoder(self.encoder(model_input))
        self.model = Model(model_input, model_output, name="autoencoder")

    def _build_decoder(self):
        decoder_input = self._add_decoder_input()
        dense_layer = self._add_dense_layer(decoder_input)
        reshape_layer = self._add_reshape_layer(dense_layer)
        conv_transpose_layers = self._add_conv_transpose_layers(reshape_layer)
        decoder_output = self._add_decoder_output(conv_transpose_layers)
        self.decoder = Model(decoder_input, decoder_output, name="decoder")

    def _add_decoder_input(self):
        return Input(shape=self.music_latent_space_dim + self.timbre_latent_space_dim, name="decoder_input")

    def _add_dense_layer(self, decoder_input):
        num_neurons = np.prod(self._shape_before_bottleneck) # [1, 2, 4] -> 8
        dense_layer = Dense(num_neurons, name="decoder_dense")(decoder_input)
        return dense_layer

    def _add_reshape_layer(self, dense_layer):
        return Reshape(self._shape_before_bottleneck)(dense_layer)

    def _add_conv_transpose_layers(self, x):
        """Add conv transpose blocks."""
        # loop through all the conv layers in reverse order and stop at the
        # first layer
        for layer_index in reversed(range(1, self._num_conv_layers)):
            x = self._add_conv_transpose_layer(layer_index, x)
        return x

    def _add_conv_transpose_layer(self, layer_index, x):
        layer_num = self._num_conv_layers - layer_index
        conv_transpose_layer = Conv2DTranspose(
            filters=self.conv_filters[layer_index],
            kernel_size=self.conv_kernels[layer_index],
            strides=self.conv_strides[layer_index],
            padding="same",
            name=f"decoder_conv_transpose_layer_{layer_num}"
        )
        x = conv_transpose_layer(x)
        x = ReLU(name=f"decoder_relu_{layer_num}")(x)
        x = BatchNormalization(name=f"decoder_bn_{layer_num}")(x)
        return x

    def _add_decoder_output(self, x):
        conv_transpose_layer = Conv2DTranspose(
            filters=1,
            kernel_size=self.conv_kernels[0],
            strides=self.conv_strides[0],
            padding="same",
            name=f"decoder_conv_transpose_layer_{self._num_conv_layers}"
        )
        x = conv_transpose_layer(x)
        output_layer = Activation("sigmoid", name="sigmoid_layer")(x)
        return output_layer

    def _build_encoder(self):
        timbre_input = self._add_timbre_input()
        music_input = self._add_music_input()

        conv_layers = self._add_conv_layers(timbre_input)
        # Join here
        timbre_bottleneck = self._add_bottleneck(conv_layers)
        bottleneck = concatenate([timbre_bottleneck, music_input])
        self._model_input = [timbre_input, music_input]
        self.encoder = Model(inputs=self._model_input, outputs=bottleneck, name="encoder")

    def _add_timbre_input(self):
        return Input(shape=self.timbre_input_shape, name="timbre_input")

    def _add_music_input(self):
        return Input(shape=self.music_input_shape, name="music_input")

    def _add_conv_layers(self, encoder_input):
        """Create all convolutional blocks in encoder."""
        x = encoder_input
        for layer_index in range(self._num_conv_layers):
            x = self._add_conv_layer(layer_index, x)
        return x

    def _add_conv_layer(self, layer_index, x):
        """Add a convolutional block to a graph of layers, consisting of
        conv 2d + ReLU + batch normalization.
        """
        layer_number = layer_index + 1
        conv_layer = Conv2D(
            filters=self.conv_filters[layer_index],
            kernel_size=self.conv_kernels[layer_index],
            strides=self.conv_strides[layer_index],
            padding="same",
            name=f"encoder_conv_layer_{layer_number}"
        )
        x = conv_layer(x)
        x = ReLU(name=f"encoder_relu_{layer_number}")(x)
        x = BatchNormalization(name=f"encoder_bn_{layer_number}")(x)
        return x

    def _add_bottleneck(self, x):
        """Flatten data and add bottleneck with Guassian sampling (Dense
        layer).
        """
        self._shape_before_bottleneck = K.int_shape(x)[1:]
        x = Flatten()(x)
        self.mu = Dense(self.timbre_latent_space_dim, name="mu")(x)
        self.log_variance = Dense(self.timbre_latent_space_dim,
                                  name="log_variance")(x)

        def sample_point_from_normal_distribution(args):
            mu, log_variance = args
            epsilon = K.random_normal(shape=K.shape(self.mu), mean=0.,
                                      stddev=1.)
            sampled_point = mu + K.exp(log_variance / 2) * epsilon
            return sampled_point

        x = Lambda(sample_point_from_normal_distribution,
                   name="encoder_output")([self.mu, self.log_variance])
        return x


2.4.0


## Data Generator

### Data generator code

In [4]:
import re
import os
import numpy as np

# Number of Reamped Clips (ampids): 80
# Number of (Sub)clips each Reamped Clip(clipids): 12
# Number of windows each Clip(windowid): different for clip:
#     1: 13
#     2: 12
#     3: 13
#     4: 11
#     5: 11
#     6: 14
#     7: 11
#     8: 14
#     9: 11
#     10: 08
#     11: 09
#     12: 15

# Null Spectrograms/Windows. To be disregarded while training
# 01-12
# 04-10
# 06-00
# 07-10
# 08-13
# 11-08
# 12-14

def get_valid_files(spectrograms_path, ampids, clipids):

    # Error checking
    # ampid check: 1-80
    invalidampids = [i for i in ampids if i>80 or i<1]
    if len(invalidampids) > 0:
        raise Exception("Invalid ampids: " + str(invalidampids))
    # clipid check: 1-12
    invalidclipids = [i for i in clipids if i>12 or i<1]
    if len(invalidclipids) > 0:
        raise Exception("Invalid clipids: " + str(invalidclipids))

    clips_to_remove = ["01-12", "04-10", "06-00", "07-10", "08-13", "11-08", "12-14"]
    # Actual DS creation
    for root, _, filenames in os.walk(spectrograms_path):

        ds = []
        for filename in filenames:
            fn_split = filename.split(" ")[0].split("-")

            ampid = int(fn_split[0])
            clipid = int(fn_split[1])
            windowid = int(fn_split[2])

            regex = "^(?!00000.*$).*"
            if re.match(regex, filename) and (ampid in ampids and clipid in clipids) and ("%02d" % clipid + "-" + "%02d" % windowid) not in clips_to_remove:
                ds.append(filename)
    return sorted(ds)

def get_music_embeddings_dict(model_path, embeddings_file, filenames_file):
    filenames = []
    with open(os.path.join(model_path, filenames_file), 'r') as f:
        r = csv.reader(f, delimiter='\t')
        for row in r:
            filenames.append(row[0])
    vec = []
    with open(os.path.join(model_path, embeddings_file), 'r') as f:
        r = csv.reader(f, delimiter='\t')
        for row in r:
            float_row = [float(i) for i in row]
            vec.append(float_row)

    d_name_vec = {}
    for i in range(0, len(filenames)):
        d_name_vec[filenames[i]] = vec[i]

    return d_name_vec

def music_ds_generator(d_name_vec, spectrograms_path, filenames, batchsize, return_filenames=False):
        i = 0
        dslen = len(filenames)
        while True:
            x, mvecs, xfiles = [], [], []

            st = i
            end = (i + batchsize) % dslen
            i = (end ) % dslen
            
            if end < st:
                xfiles = filenames[st:dslen]
                xfiles.extend(filenames[0:end])
            else:
                xfiles = filenames[st: end]
        
            for filename in xfiles:
                filepath = os.path.join(spectrograms_path, filename)
                
                spectrogram = np.load(filepath)
                mvec = d_name_vec[filename]
                
                x.append(spectrogram[..., np.newaxis])
                mvecs.append(mvec)
            if return_filenames:
                yield (((np.array(x), np.array(mvecs)), np.array(x)), xfiles)
            else:
                yield ((np.array(x), np.array(mvecs)), np.array(x))


### Driver for data generator

In [42]:
SPECTROGRAMS_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/spectrogram"
TIMBREAE_SAVE_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/timbre-encoder/"
MUSICAE_SAVE_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/music-encoder/"

TIMBREAE_MODEL_PATH = os.path.join(TIMBREAE_SAVE_PATH, "model")
MUSICAE_MODEL_PATH = os.path.join(MUSICAE_SAVE_PATH, "model")

EMBEDDINGS_FILE = "embeddings-2021-10-04-06-08-29.tsv"
FILENAMES_FILE = "embedding-filenames-2021-10-04-06-08-29.tsv"

AMP_IDS = [i for i in range(1, 81)] # 1-80
CLIP_IDS = [i for i in range(1, 13)] # 1-12
BATCH_SIZE = 27

dsfilenames = get_valid_files(SPECTROGRAMS_PATH, AMP_IDS, CLIP_IDS)
dslen = len(dsfilenames)
print("Total dataset size: " + str(dslen))

d_name_vec = get_music_embeddings_dict(MUSICAE_MODEL_PATH, EMBEDDINGS_FILE, FILENAMES_FILE)

musicds_gen = music_ds_generator(d_name_vec, SPECTROGRAMS_PATH, dsfilenames, BATCH_SIZE, False)

Total dataset size: 10800


In [ ]:
i =0
for m in musicds_gen:
    print(np.shape(m[0][0]))    
    print(np.shape(m[0][1]))

## Timbre AutoEncoder

### Instantiate Or Load

In [43]:
TIMBREAE_SAVE_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/timbre-encoder/"
TIMBREAE_MODEL_PATH = os.path.join(TIMBREAE_SAVE_PATH, "model")
LEARNING_RATE = 0.0001

timbreae = VAE.load(TIMBREAE_MODEL_PATH, 'weights.h5')

if timbreae == None:
    print("Instantiating the model")
    MUSIC_LATENT_SPACE_DIM= 16
    TIMBRE_LATENT_SPACE_DIM= 4
    CONV_FILTERS = (128, 64, 32, 32)
    CONV_KERNELS = (3, 3, 3, 3)
    CONV_STRIDES = (1, 2, 2, 1)

    timbreae = VAE(
        timbre_input_shape= (256, 64, 1),
        music_input_shape= (16, ),
        conv_filters= CONV_FILTERS, 
        conv_kernels= CONV_KERNELS,
        conv_strides= CONV_STRIDES,
        music_latent_space_dim=MUSIC_LATENT_SPACE_DIM,
        timbre_latent_space_dim=TIMBRE_LATENT_SPACE_DIM
    )
    timbreae.save_parameters(TIMBREAE_MODEL_PATH)

timbreae.compile(LEARNING_RATE)
timbreae.summary()

Instantiating the model
[(256, 64, 1), (16,), (128, 64, 32, 32), (3, 3, 3, 3), (1, 2, 2, 1), 16, 4]
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
timbre_input (InputLayer)       [(None, 256, 64, 1)] 0                                            
__________________________________________________________________________________________________
encoder_conv_layer_1 (Conv2D)   (None, 256, 64, 128) 1280        timbre_input[0][0]               
__________________________________________________________________________________________________
encoder_relu_1 (ReLU)           (None, 256, 64, 128) 0           encoder_conv_layer_1[0][0]       
__________________________________________________________________________________________________
encoder_bn_1 (BatchNormalizatio (None, 256, 64, 128) 512         encoder_relu_1[0][0]      

### Train Model (Iterative)

In [ ]:
import pytz
from datetime import datetime

EPOCHS = 500
TIMBREAE_SAVE_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/timbre-encoder/"
TIMBREAE_MODEL_PATH = os.path.join(TIMBREAE_SAVE_PATH, "model")

ts = pytz.timezone('Asia/Tokyo').localize(datetime.now())
tsf = ts.strftime("%Y-%m-%d-%H-%M-%S")
CHECKPOINT_PATH = os.path.join(TIMBREAE_MODEL_PATH, "checkpoints-" + tsf)

if not os.path.exists(CHECKPOINT_PATH):
    os.makedirs(CHECKPOINT_PATH)

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        save_checkpoint_path = os.path.join(CHECKPOINT_PATH, "weights-{}.h5".format(epoch))
        save_path = os.path.join(TIMBREAE_MODEL_PATH, "weights.h5")
        self.model.save_weights(save_checkpoint_path)
        self.model.save_weights(save_path)

history = timbreae.train(
    generator=musicds_gen, 
    dataset_length=dslen,
    batch_size=BATCH_SIZE, 
    num_epochs=EPOCHS,
    callbacks=[CustomCallback()]
    )
losses = history.history['loss']


Epoch 1/500
400/400 [==============================] - 41s 103ms/step - batch: 199.5000 - size: 27.0000 - loss: 8706.8106 - _calculate_reconstruction_loss: 0.0084 - _calculate_kl_loss: 257.4191
Epoch 2/500
400/400 [==============================] - 41s 102ms/step - batch: 199.5000 - size: 27.0000 - loss: 8349.6759 - _calculate_reconstruction_loss: 0.0081 - _calculate_kl_loss: 243.3278
Epoch 3/500
400/400 [==============================] - 41s 102ms/step - batch: 199.5000 - size: 27.0000 - loss: 8531.2483 - _calculate_reconstruction_loss: 0.0083 - _calculate_kl_loss: 246.1919
Epoch 4/500
400/400 [==============================] - 41s 102ms/step - batch: 199.5000 - size: 27.0000 - loss: 9254.6054 - _calculate_reconstruction_loss: 0.0089 - _calculate_kl_loss: 321.4472
Epoch 5/500
400/400 [==============================] - 41s 102ms/step - batch: 199.5000 - size: 27.0000 - loss: 8571.4451 - _calculate_reconstruction_loss: 0.0083 - _calculate_kl_loss: 286.8864
Epoch 6/500
400/400 [=========

### Save Meta

In [46]:
# Training History
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd


gc = gspread.authorize(GoogleCredentials.get_application_default())
ws = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ipVFC30pOjxDpsIrSqPiYxqGhbXiLjsFSBajyzo0gew/edit#gid=1946289387').sheet1
all = ws.get_all_records()
last_ts = all[-1]['Timestamp']
ws.resize(len(all)+1)
if last_ts != ts.strftime("%Y/%m/%d %H:%M:%S"):
    ws.append_row([
                  ts.strftime("%Y/%m/%d %H:%M:%S"),
                  str(AMP_IDS),
                  str(CLIP_IDS),
                  str(CONV_FILTERS),
                  str(CONV_KERNELS),
                  str(CONV_STRIDES),
                  str(TIMBRE_LATENT_SPACE_DIM),
                  str(LEARNING_RATE),
                  str(BATCH_SIZE),
                  str(EPOCHS),
                  str(int(min(losses))),
                  str(losses)
    ])
else:
    print("Already added to Training history!")

print("Metadata Saved!!")



Metadata Saved!!


## Generate the embeddings

In [ ]:
# # https://github.com/musikalkemist/generating-sound-with-neural-networks/blob/49d7db32c43d1a04c596cbbb282a9521be1e7fc8/11%20Implementing%20VAE/code/analysis.py

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display


# Same method as above
def load_fsdd(spectrograms_path):
    dataset = {}
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:

            if True: # re.match(regex, file_name):
                file_path = os.path.join(root, file_name)
                spectrogram = np.load(file_path) # (n_bins, n_frames, 1) 
                dataset[file_name] = spectrogram[..., np.newaxis]
    return dataset
SPECTROGRAMS_PATH = "/content/drive/MyDrive/Music/VAE/spectrogram"

LATENT_REPRESENTATIONS_PATH = '/content/drive/MyDrive/Music/VAE/model-' + str(LATENT_SPACE_DIM) + '-' + str(BATCH_SIZE) + '-' + str(EPOCHS) + '/'
MODEL_PATH = '/content/drive/MyDrive/Music/VAE/model-8-50-100'

def select_random_images(dataset, num_images=10):
    sample_keys = np.random.choice(list(dataset.keys()), num_images)
    sample_ds = { key: dataset[key] for key in sample_keys }

    return sample_ds


def plot_reconstructed_images(images, reconstructed_images):
    num_images = len(images)
    for i, (image, reconstructed_image) in enumerate(zip(images, reconstructed_images)):

        fig, ax = plt.subplots()
        image = image.squeeze()
        img = librosa.display.specshow(image, y_axis='log', x_axis='time', ax=ax)
        fig.colorbar(img, ax=ax, format="%+2.0f dB")

        fig, ax = plt.subplots()
        reconstructed_image = reconstructed_image.squeeze()
        recon_img = librosa.display.specshow(reconstructed_image, y_axis='log', x_axis='time', ax=ax)
        fig.colorbar(recon_img, ax=ax, format="%+2.0f dB")
    plt.show()


def plot_images_encoded_in_latent_space(latent_representations, sample_labels):
    plt.figure(figsize=(10, 10))
    plt.scatter(latent_representations[:, 0],
                latent_representations[:, 1],
                cmap="rainbow",
                c=sample_labels,
                alpha=0.5,
                s=2)
    plt.colorbar()
    plt.show()

def download_vectors(autoencoder, sample_ds, download_path):
    images = np.array(list(sample_ds.values()))
    filenames = sample_ds.keys()
    latent_representations = autoencoder.encoder.predict(images)

    with open(download_path + 'embeddings.tsv', 'w', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerows(latent_representations)
    # Write corresdonding filenames
    with open(download_path + 'embedding-filenames.tsv', 'a') as f_output:
        f_output.seek(0)
        f_output.truncate()
        for data in filenames:
            f_output.write(data)
            f_output.write('\n')
    return latent_representations, filenames


autoencoder = VAE.load(MODEL_PATH)
dataset = load_fsdd(SPECTROGRAMS_PATH)

# num_sample_images_to_show = 30

# sample_ds = select_random_images(dataset, num_sample_images_to_show)
# reconstructed_images, _ = autoencoder.reconstruct(np.array(list(dataset.values())))
download_vectors(autoencoder, dataset, LATENT_REPRESENTATIONS_PATH)

# plot_reconstructed_images(sample_images, reconstructed_images)

# num_images = 6000
# sample_images, sample_labels = select_images(x_test, y_test, num_images)
# _, latent_representations = autoencoder.reconstruct(sample_images)
# plot_images_encoded_in_latent_space(latent_representations, sample_labels)

### Load the Data generator

In [5]:
SPECTROGRAMS_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/spectrogram"
MUSICAE_SAVE_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/music-encoder/"

MUSICAE_MODEL_PATH = os.path.join(MUSICAE_SAVE_PATH, "model")

EMBEDDINGS_FILE = "embeddings-2021-10-04-06-08-29.tsv"
FILENAMES_FILE = "embedding-filenames-2021-10-04-06-08-29.tsv"

AMP_IDS = [i for i in range(1, 81)] # 1-80
CLIP_IDS = [i for i in range(1, 13)] # 1-12
BATCH_SIZE = 27

dsfilenames = get_valid_files(SPECTROGRAMS_PATH, AMP_IDS, CLIP_IDS)
dslen = len(dsfilenames)
print("Total dataset size: " + str(dslen))

d_name_vec = get_music_embeddings_dict(MUSICAE_MODEL_PATH, EMBEDDINGS_FILE, FILENAMES_FILE)

musicds_gen = music_ds_generator(d_name_vec, SPECTROGRAMS_PATH, dsfilenames, BATCH_SIZE, False)

Total dataset size: 10800


### Load the Model

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/music-encoder/model"
WEIGHTS_FILE = "weights.h5"
LEARNING_RATE = 0.0001

# TODO before running this cell independently
# 1. Update the WEIGHTS_FILE

musicae = VAE.load(MODEL_PATH, WEIGHTS_FILE)

musicae.compile(LEARNING_RATE)
musicae.summary()

### Generate Embeddings

In [ ]:
# # https://github.com/musikalkemist/generating-sound-with-neural-networks/blob/49d7db32c43d1a04c596cbbb282a9521be1e7fc8/11%20Implementing%20VAE/code/analysis.py

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pytz
from datetime import datetime

ts = pytz.timezone('Asia/Tokyo').localize(datetime.now())
tsf = ts.strftime("%Y-%m-%d-%H-%M-%S")

SPECTROGRAMS_PATH = "/content/drive/MyDrive/Music/VAE/ICASSP/spectrogram"


def save_embeddings(musicae, dspath, dsfilenames, download_path, tsf):

    dsspectrogram = []
    for filename in dsfilenames:
        filepath = os.path.join(SPECTROGRAMS_PATH, filename)
        spectrogram = np.load(filepath)
        dsspectrogram.append(spectrogram[..., np.newaxis])

    dsspectrogram = np.array(dsspectrogram)
    latent_representations = musicae.encoder.predict(dsspectrogram)

    with open(os.path.join(download_path, 'embeddings-' + tsf + '.tsv'), 'a', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerows(latent_representations)
    # Write corresdonding filenames
    with open(os.path.join(download_path, 'embedding-filenames-' + tsf + '.tsv'), 'a') as f_output:
        # f_output.seek(0)
        # f_output.truncate()
        for data in dsfilenames:
            f_output.write(data)
            f_output.write('\n')
    print("Embeddings saved!!")
    return latent_representations

## Driver coder
times = 0
for _, _, xfiles, _ in musicds_gen:
    print(xfiles)
    save_embeddings(musicae, SPECTROGRAMS_PATH, xfiles, MODEL_PATH, tsf)
    times += 1
    if times == dslen/BATCH_SIZE:
        break

# reconstructed_images, _ = autoencoder.reconstruct(np.array(list(dataset.values())))
# plot_reconstructed_images(sample_images, reconstructed_images)

### testing

In [21]:
import os
import pickle
import csv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Flatten, Dense, Reshape, Conv2DTranspose, Activation, Lambda, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import numpy as np
import tensorflow as tf


In [27]:
musicinput = Input(shape=(16, ), name="musicinput")
timbreinput = Input(shape=(256, 64, ), name="timbreinput")
flatteninput = Flatten()(timbreinput)
timbreoutput = Dense(4)(flatteninput)
print(timbreoutput.get_shape())
output = concatenate([timbreoutput, musicinput])  # merge the outputs of the two models

model = Model(inputs=[timbreinput, musicinput], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

music_input = np.random.random((128, 16))
timbre_input = np.random.random((128, 256, 64))

# timbre_output = np.random.random((128, 4))
# model.fit(x=[timbre_input, music_input], y=[timbre_output, music_input])

timbre_embedding = np.random.random((1, 256, 64))
music_embedding = np.random.random((1, 16))
print(music_embedding)
model.predict((timbre_embedding, music_embedding))

(None, 4)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
timbreinput (InputLayer)        [(None, 256, 64)]    0                                            
__________________________________________________________________________________________________
flatten_7 (Flatten)             (None, 16384)        0           timbreinput[0][0]                
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 4)            65540       flatten_7[0][0]                  
__________________________________________________________________________________________________
musicinput (InputLayer)         [(None, 16)]         0                                            
__________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


array([[ 0.0071329 , -0.12648135, -0.7631175 ,  0.7477544 ,  0.7323161 ,
         0.6496016 ,  0.15110146,  0.37318686,  0.16791524,  0.05101025,
         0.66146106,  0.10814263,  0.65915585,  0.5141086 ,  0.37690058,
         0.59756434,  0.02245474,  0.8708205 ,  0.17650428,  0.2318658 ]],
      dtype=float32)

In [18]:
model.predict(np.random.random((1, 32)))

array([[-0.254264]], dtype=float32)